In [48]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_excel("/Work/Open.ru/OTP_session_id.xlsx")

In [50]:
df

,client_id,sessionId,datetime,status,category
0,237723095919,1af2a994-0039-4d92-bb78-c217a50ab82a,2021-05-04,CORRECT,default
1,237650241603,148b3ffb-ee4c-4f5b-9098-436993a0ade0,2021-05-04,CORRECT,default
2,237613554852,84a0ac11-f876-4e16-9b3d-b548f809eb46,2021-05-16,CORRECT,otp-login
3,237412528821,79396f39-b31a-465b-91f6-1c7f2a08dfe8,2021-05-04,CORRECT,operation-name_web
4,237178873863,c624f6cd-54ef-4099-b433-cd80f4071301,2021-05-04,CORRECT,otp-login
...,...,...,...,...,...
10159,237322368145,e8f5331-6349-47d5-ac73-495a287fd16d,2021-06-17,CORRECT,otp-login
10160,239613229075,35edc5b-b0bd-4eb5-8d39-c880ece9dab8,2021-06-17,CORRECT,otp-login
10161,237107862973,d64fa9a-71f9-418d-a862-ea48bb8f0235,2021-06-17,CORRECT,otp-login
10162,237411479323,f276d6d-8ac9-4765-a4c0-565736d41a95,2021-06-17,INCORRECT,default


Задание 1.
Добавить индикатор, который будет выделять следующие значения:
•	Если otp для категории платежей (payment), то «1»
•	Если otp для категории переводов (transfer), но не для переводов с использованием счетов (acc), то «2»
Все остальные заявки не должны попасть в результат выполнения запроса.


In [51]:
dt = df.copy()

In [52]:
conditions =[
    (dt['category'].str.contains(r"payment")),
    (dt['category'].str.contains(r"transfer"))&(~dt['category'].str.contains(r"acc"))
]
choices = [1,2]

dt['indicator'] = np.select(conditions,choices, default='not_best')
dt.query('indicator == ("1","2")')


,client_id,sessionId,datetime,status,category,indicator
6,238519442889,7fb5144e-458f-4556-a624-c2c4b6920b3f,2021-05-04,CORRECT,transfer_card_web,2
17,237788391768,f8fb0b85-e525-4fcf-b242-791c35a3c20b,2021-05-04,CORRECT,payment_card_web,1
32,237097828287,2b2d3d29-d35d-402e-bddd-028113928736,2021-05-04,CORRECT,transfer-p2p_card_web,2
45,237245912013,d321701a-60f9-455b-a60a-7b62919ead6b,2021-05-04,CORRECT,payment_card_web,1
57,237683448462,8832ef3a-b9ec-47e6-aa02-0f697f684815,2021-05-04,CORRECT,payment_card_web,1
...,...,...,...,...,...,...
10135,237586095034,8ba90f4-a806-42e2-a554-94b972fe78cd,2021-06-17,CORRECT,payment_card_web,1
10140,237440193496,46b3426-0ca5-4e6c-9ee8-24f00b2d5b92,2021-06-17,CORRECT,payment_card_web,1
10147,237095861815,78583b7-18fc-4afa-b3d4-03b705aee69f,2021-06-17,CORRECT,payment_card_web,1
10152,237417392068,05bd069-b795-46fb-a6b8-d36baa3fdaf4,2021-06-17,CORRECT,payment_card_web,1


Задание 2. 
Посчитать метрику Month-of-Month (прирост текущего месяца к предыдущему) по уникальным клиентам с кодами otp-login.

In [86]:
dt2 = df.copy()

In [87]:
dt2 = dt2.query('category == "otp-login"')

In [88]:
dt2 = dt2.drop_duplicates(subset='client_id', keep="first")

In [89]:
dt2['month'] = dt2['datetime'].dt.month

In [90]:
from itertools import count
dt2 = dt2.groupby('month')['client_id'].agg(['count'])

In [91]:
dt2 = dt2.reset_index()

In [92]:
dt22 = dt2.copy()

In [93]:
dt22

,month,count
0,5,2529
1,6,1394


In [84]:
dt22 = dt22.reset_index()

In [94]:

dt22['growth percentage diff'] = np.round((dt22['count'].div(dt22['count'].shift(1)) - 1),3)
dt22

,month,count,growth percentage diff
0,5,2529,NaN
1,6,1394,-0.449



Задание 3.
Одним запросом сформируйте:
•	Количество успешно введённых ОТП кодов в разрезе категории кода ОТП
•	Долю каждой категории по убыванию
•	Количество с накопительным итогом
•	Общее количество введённых ОТП кодов
•	Последний отчётный месяц (полный месяц, от текущего)


In [95]:
dt3 = df.copy()

In [96]:
dt4 = df.copy()

In [97]:
dt3.head()

,client_id,sessionId,datetime,status,category
0,237723095919,1af2a994-0039-4d92-bb78-c217a50ab82a,2021-05-04,CORRECT,default
1,237650241603,148b3ffb-ee4c-4f5b-9098-436993a0ade0,2021-05-04,CORRECT,default
2,237613554852,84a0ac11-f876-4e16-9b3d-b548f809eb46,2021-05-16,CORRECT,otp-login
3,237412528821,79396f39-b31a-465b-91f6-1c7f2a08dfe8,2021-05-04,CORRECT,operation-name_web
4,237178873863,c624f6cd-54ef-4099-b433-cd80f4071301,2021-05-04,CORRECT,otp-login


In [98]:
dt3 = dt3.query('status == "CORRECT"')

In [99]:
dt3 = dt3['category'].value_counts(dropna=False)


In [100]:
dt3 = pd.DataFrame(dt3)

In [101]:
dt3 = dt3.reset_index()

In [102]:
dt3.columns = ['category','success']

In [103]:
dt3

,category,success
0,otp-login,5497
1,default,1979
2,payment_card_web,1207
3,operation-name_web,621
4,transfer-p2p_card_web,217
5,transfer_card_web,117
6,transfer-free_acc_web,115
7,sms-info_off_web,32
8,credit-contract-sign_web,11
9,limit_set_web,8


Задание 3.  Одним запросом сформируйте:
•	Количество успешно введённых ОТП кодов в разрезе категории кода ОТП
•	Долю каждой категории по убыванию
•	Количество с накопительным итогом
•	Общее количество введённых ОТП кодов
•	Последний отчётный месяц (полный месяц, от текущего)

In [104]:
dt3['share'] = np.round((dt3['success'] / dt3['success'].sum()) * 100,0).astype(str) + '%'

In [105]:
dt3

,category,success,share
0,otp-login,5497,56.0%
1,default,1979,20.0%
2,payment_card_web,1207,12.0%
3,operation-name_web,621,6.0%
4,transfer-p2p_card_web,217,2.0%
5,transfer_card_web,117,1.0%
6,transfer-free_acc_web,115,1.0%
7,sms-info_off_web,32,0.0%
8,credit-contract-sign_web,11,0.0%
9,limit_set_web,8,0.0%


In [106]:
dt3['cumulative_total'] = dt3['success'].cumsum()

In [107]:
dt3

,category,success,share,cumulative_total
0,otp-login,5497,56.0%,5497
1,default,1979,20.0%,7476
2,payment_card_web,1207,12.0%,8683
3,operation-name_web,621,6.0%,9304
4,transfer-p2p_card_web,217,2.0%,9521
5,transfer_card_web,117,1.0%,9638
6,transfer-free_acc_web,115,1.0%,9753
7,sms-info_off_web,32,0.0%,9785
8,credit-contract-sign_web,11,0.0%,9796
9,limit_set_web,8,0.0%,9804


In [108]:
dt3['total_otp_count'] = dt4['status'].count()

In [109]:
dt3

,category,success,share,cumulative_total,total_otp_count
0,otp-login,5497,56.0%,5497,10164
1,default,1979,20.0%,7476,10164
2,payment_card_web,1207,12.0%,8683,10164
3,operation-name_web,621,6.0%,9304,10164
4,transfer-p2p_card_web,217,2.0%,9521,10164
5,transfer_card_web,117,1.0%,9638,10164
6,transfer-free_acc_web,115,1.0%,9753,10164
7,sms-info_off_web,32,0.0%,9785,10164
8,credit-contract-sign_web,11,0.0%,9796,10164
9,limit_set_web,8,0.0%,9804,10164


In [110]:
from datetime import date, timedelta
dt3['last_month'] =  date.today().replace(day=1) - timedelta(days=1)

In [111]:
dt3

,category,success,share,cumulative_total,total_otp_count,last_month
0,otp-login,5497,56.0%,5497,10164,2022-06-30
1,default,1979,20.0%,7476,10164,2022-06-30
2,payment_card_web,1207,12.0%,8683,10164,2022-06-30
3,operation-name_web,621,6.0%,9304,10164,2022-06-30
4,transfer-p2p_card_web,217,2.0%,9521,10164,2022-06-30
5,transfer_card_web,117,1.0%,9638,10164,2022-06-30
6,transfer-free_acc_web,115,1.0%,9753,10164,2022-06-30
7,sms-info_off_web,32,0.0%,9785,10164,2022-06-30
8,credit-contract-sign_web,11,0.0%,9796,10164,2022-06-30
9,limit_set_web,8,0.0%,9804,10164,2022-06-30
